In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')


In [3]:
# Separate target variable
y_train = train_data['SalePrice']


In [4]:
# Concatenate train and test data for preprocessing
all_data = pd.concat([train_data.drop(columns=['SalePrice']), test_data], axis=0, ignore_index=True)

In [5]:
# Identify numerical and categorical columns
numerical_cols = all_data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = all_data.select_dtypes(include=['object']).columns

In [6]:
# Preprocessing pipelines for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', StandardScaler())  # Standardize numerical features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

In [7]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [8]:
# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [9]:
# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])


In [10]:
# Split data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(train_data.drop(columns=['SalePrice']), y_train,
                                                      train_size=0.8, test_size=0.2, random_state=0)

In [11]:
# Fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF'...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('model', RandomForestRegressor(random_state=0))])

In [12]:
# Predict on validation set
preds_valid = pipeline.predict(X_valid)

In [13]:
# Evaluate model
print('Mean Squared Error:', mean_squared_error(y_valid, preds_valid, squared=False))

Mean Squared Error: 34510.919861651186


In [14]:
# Example of predicting on test data
test_preds = pipeline.predict(test_data)

In [15]:
# Prepare submission file for Kaggle
submission = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': test_preds})
submission.to_csv('submission.csv', index=False)
print('Submission file saved.')


Submission file saved.
